In [1]:
import os
import re
import numpy as np
import pandas as pd
from utils.helper_functions import find_files_in_dir, sleep_metrics, calculate_cbtmin

# Chang14 simulations

In [7]:
simulation_condition_names = [
    "sleep_parameters", "circadian_parameters",
]
chang14_save_paths = [
    "../data/simulations/chang14_sleep_parameters",
    "../data/simulations/chang14_circadian_parameters",
]
baseline_lux_values = [90, 500]
bedtime = 22.0
ebook_order_options = ["ebook_first", "ebook_second"]
time = np.load("../data/light_schedules/time.npy")
daily_condition_options = {
    'ebook_first': [
        'None',
        'eBook', 'eBook', 'eBook', 'eBook', 'eBook',
        'None',
        'Book', 'Book', 'Book', 'Book', 'Book',
        'None',
    ],
    'ebook_second': [
        'None',
        'Book', 'Book', 'Book', 'Book', 'Book',
        'None',
        'eBook', 'eBook', 'eBook', 'eBook', 'eBook',
        'None',
    ],
}

In [11]:
for jdx in range(len(simulation_condition_names)):
    results = pd.DataFrame()
    simulation_condition = simulation_condition_names[jdx]
    save_path = chang14_save_paths[jdx]
    for kdx in range(len(baseline_lux_values)):
        baseline_lux = baseline_lux_values[kdx]
        for ldx in range(len(ebook_order_options)):
            ebook_order = ebook_order_options[ldx]
            pattern = re.compile(
                rf"baseline_lux_{baseline_lux}_{ebook_order}_simulation_(\d+).npz"
            )
            files = find_files_in_dir(save_path, pattern)
            for filename in files:
                simulation_idx = int(re.search(pattern, filename).group(1))
                data = np.load(filename, allow_pickle=True)
                sleep = data["sleep"]
                states = data["trajectory_states"]
                parameters = data["parameters"]
                sleep_duration, sleep_onset, _ = sleep_metrics(time, sleep)
                cbtmin = calculate_cbtmin(time, states)

                min_length = min(
                    len(sleep_duration), len(sleep_onset), len(cbtmin)
                )
                sleep_duration = sleep_duration[:min_length]
                sleep_onset = sleep_onset[:min_length]
                cbtmin = cbtmin[:min_length]

                days = np.arange(1, min_length + 1)

                normalized_sleep_onset = sleep_onset - days * 24 + 24
                normalized_cbtmin = cbtmin - days * 24 + 24
                normalized_sleep_latency = (normalized_sleep_onset - bedtime) * 60
                normalized_sleep_latency[normalized_sleep_latency < 0] = 0.0

                results = pd.concat([results, pd.DataFrame({
                    'simulation_idx': [simulation_idx] * min_length,
                    'day': days,
                    'simulation_condition': [ebook_order] * min_length,
                    'sleep_onset': normalized_sleep_onset,
                    'sleep_duration': sleep_duration,
                    'cbtmin': normalized_cbtmin,
                    'sleep_latency_minutes': normalized_sleep_latency,
                    'daily_condition': daily_condition_options[ebook_order][:min_length],
                    'baseline_lux': [baseline_lux] * min_length,
                })])
    results.to_csv(f"{save_path}/chang14_{simulation_condition}_features.csv", index=False)
    print(f"Saved {save_path}/chang14_{simulation_condition}_features.csv")

Saved ../data/simulations/chang14_sleep_parameters/chang14_sleep_parameters_features.csv
Saved ../data/simulations/chang14_circadian_parameters/chang14_circadian_parameters_features.csv


# Different initial conditions

In [7]:
simulation_condition_names = [
    "different_initial_conditions"
]
chang14_save_paths = [
    "../data/simulations/different_initial_conditions",
]
baseline_lux_values = [90, 500]
lights_on_times = [4, 5, 6, 7, 8, 9, 10, 11]
bedtime = 22.0
ebook_order_options = ["ebook_first", "ebook_second"]
time = np.load("../data/light_schedules/time.npy")
daily_condition_options = {
    'ebook_first': [
        'None',
        'eBook', 'eBook', 'eBook', 'eBook', 'eBook',
        'None',
        'Book', 'Book', 'Book', 'Book', 'Book',
        'None',
    ],
    'ebook_second': [
        'None',
        'Book', 'Book', 'Book', 'Book', 'Book',
        'None',
        'eBook', 'eBook', 'eBook', 'eBook', 'eBook',
        'None',
    ],
}

In [3]:
for jdx in range(len(simulation_condition_names)):
    results = pd.DataFrame()
    simulation_condition = simulation_condition_names[jdx]
    save_path = chang14_save_paths[jdx]
    for kdx in range(len(baseline_lux_values)):
        baseline_lux = baseline_lux_values[kdx]
        for ldx in range(len(ebook_order_options)):
            ebook_order = ebook_order_options[ldx]
            pattern = re.compile(
                rf"baseline_lux_{baseline_lux}_{ebook_order}_simulation_lights_on_(\d+).npz"
            )
            files = find_files_in_dir(save_path, pattern)
            for filename in files:
                lights_on = int(re.search(pattern, filename).group(1))
                data = np.load(filename, allow_pickle=True)
                sleep = data["sleep"]
                states = data["trajectory_states"]
                parameters = data["parameters"]
                sleep_duration, sleep_onset, _ = sleep_metrics(time, sleep)
                cbtmin = calculate_cbtmin(time, states)

                min_length = min(
                    len(sleep_duration), len(sleep_onset), len(cbtmin)
                )
                sleep_duration = sleep_duration[:min_length]
                sleep_onset = sleep_onset[:min_length]
                cbtmin = cbtmin[:min_length]

                days = np.arange(1, min_length + 1)

                normalized_sleep_onset = sleep_onset - days * 24 + 24
                normalized_cbtmin = cbtmin - days * 24 + 24
                normalized_sleep_latency = (normalized_sleep_onset - bedtime) * 60
                normalized_sleep_latency[normalized_sleep_latency < 0] = 0.0

                results = pd.concat([results, pd.DataFrame({
                    'lights_on': lights_on,
                    'day': days,
                    'simulation_condition': [ebook_order] * min_length,
                    'sleep_onset': normalized_sleep_onset,
                    'sleep_duration': sleep_duration,
                    'cbtmin': normalized_cbtmin,
                    'sleep_latency_minutes': normalized_sleep_latency,
                    'daily_condition': daily_condition_options[ebook_order][:min_length],
                    'baseline_lux': [baseline_lux] * min_length,
                })])
    results.to_csv(f"{save_path}/chang14_{simulation_condition}_features.csv", index=False)
    print(f"Saved {save_path}/chang14_{simulation_condition}_features.csv")

Saved ../data/simulations/different_initial_conditions/chang14_different_initial_conditions_features.csv


In [9]:
# Manually calculate phase shift for the different wake times to include in the article
for lights_on in lights_on_times:
    data = pd.read_csv(f"{chang14_save_paths[0]}/chang14_different_initial_conditions_features.csv")
    data_ebook_first = data[(data['lights_on'] == lights_on) & (data['baseline_lux'] == 90) & (data['simulation_condition'] == 'ebook_first')]
    cbtmin_ebook_for_ebook_first = data_ebook_first[(data_ebook_first['day'] == 7)]['cbtmin'].values[0] 
    cbtmin_book_for_ebook_first = data_ebook_first[(data_ebook_first['day'] == 13)]['cbtmin'].values[0]
    phase_shift_ebook_first = cbtmin_ebook_for_ebook_first - cbtmin_book_for_ebook_first

    data_ebook_second = data[(data['lights_on'] == lights_on) & (data['baseline_lux'] == 90) & (data['simulation_condition'] == 'ebook_second')]
    cbtmin_book_for_ebook_second = data_ebook_second[(data_ebook_second['day'] == 7)]['cbtmin'].values[0]
    cbtmin_ebook_for_ebook_second = data_ebook_second[(data_ebook_second['day'] == 13)]['cbtmin'].values[0]
    phase_shift_ebook_second = cbtmin_ebook_for_ebook_second - cbtmin_book_for_ebook_second

    print(f"Phase shift for lights_on {lights_on}:")
    print(f"Phase shift for ebook_first: {phase_shift_ebook_first}")
    print(f"Phase shift for ebook_second: {phase_shift_ebook_second}")

Phase shift for lights_on 4:
Phase shift for ebook_first: 0.4300000000000068
Phase shift for ebook_second: 0.9400000000000261
Phase shift for lights_on 5:
Phase shift for ebook_first: 0.6350000000000193
Phase shift for ebook_second: 0.6800000000000068
Phase shift for lights_on 6:
Phase shift for ebook_first: 0.8300000000000125
Phase shift for ebook_second: 0.4200000000000159
Phase shift for lights_on 7:
Phase shift for ebook_first: 1.0050000000000239
Phase shift for ebook_second: 0.160000000000025
Phase shift for lights_on 8:
Phase shift for ebook_first: 1.1899999999999982
Phase shift for ebook_second: -0.10500000000001819
Phase shift for lights_on 9:
Phase shift for ebook_first: 1.335000000000008
Phase shift for ebook_second: -0.3100000000000023
Phase shift for lights_on 10:
Phase shift for ebook_first: 1.4899999999999807
Phase shift for ebook_second: -0.5400000000000205
Phase shift for lights_on 11:
Phase shift for ebook_first: 1.6799999999999784
Phase shift for ebook_second: -0.8300

# Light pulse simulations

In [38]:
data_path = '../data/simulations/light_pulse_simulations'

baseline_lux_values = np.array([0, 25, 50, 75, 100, 250, 500, 750, 1000], dtype=float)
pulse_at_values = np.linspace(0, 14, 17)

simulation_days = 20
dt = 0.001
last_day_start = int((simulation_days - 1) * 24 / dt)
day_length_in_idx = int(24.0 / dt) - 1
time_array = np.arange(0, simulation_days * 24, dt)  # Time array for the entire simulation

avg_amplitude = np.zeros((len(baseline_lux_values), len(pulse_at_values)), dtype=float)

for idx, baseline_lux in enumerate(baseline_lux_values):
    for jdx, pulse_at in enumerate(pulse_at_values):
        data = np.load(f'{data_path}/baseline_{baseline_lux}_pulse_at_{pulse_at}.npz')
        sol = data['sol']
        light = data['light_history']
        # Calculate amplitude after pulse
        time_last_day = time_array[last_day_start:] - time_array[last_day_start]
        x_last_day = sol[0, last_day_start:]
        xc_last_day = sol[1, last_day_start:]
        light_last_day = light[last_day_start:]
        amplitude_last_day = np.sqrt(x_last_day ** 2 + xc_last_day ** 2)
        avg_amplitude[idx, jdx] = np.mean(amplitude_last_day)

# Save average amplitude
avg_amplitude_df = pd.DataFrame(avg_amplitude, index=baseline_lux_values, columns=pulse_at_values)
avg_amplitude_df.index.name = 'Baseline Lux'
avg_amplitude_df.columns.name = 'Pulse At (hours)'
avg_amplitude_df.to_csv(f'{data_path}/average_amplitude.csv')